In [1]:
import sys


sys.path.append("../src/")

In [20]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt


root = Path("/home/clay/research/kaggle/sennet/data_dumps/processed_2d/kidney_1_dense")
mask_paths = sorted(list(root.glob("*mask*")))
print(len(mask_paths))

114


In [24]:
for p in mask_paths:
    mask = cv2.imread(str(p), 0)
    contours, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    max_contour = 0
    for c in contours:
        a = cv2.contourArea(c)
        if a > max_area:
            max_contour = c
            max_area = a
    # print(max_area)
    if max_area < 1e-3:
        continue
    hull = cv2.convexHull(max_contour)
    new_mask = cv2.drawContours(np.zeros_like(mask), [hull], 0, 255, -1)
    cv2.imwrite(str(p), new_mask)
    # break

In [19]:
mask

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [13]:
mean_mmap_0 = read_mmap_array(chunk_path_0 / "mean_prob")
mean_mmap_1 = read_mmap_array(chunk_path_1 / "mean_prob")
np.abs(mean_mmap_0.data - mean_mmap_1.data).sum()

12448530.468581084

In [14]:
count_mmap_0 = read_mmap_array(chunk_path_0 / "total_count")
count_mmap_1 = read_mmap_array(chunk_path_1 / "total_count")
np.abs(count_mmap_0.data - count_mmap_1.data).sum()

0